## 모델선정 2
- trial_model의 시도를 바탕으로 데이터를 잘 학습하고 분류 정확도가 높은 성능이 좋은 모델을 사용할 필요성이 증대되었습니다
- KoBERT모델을 이용하여 학습을 진행하였습니다. 모델 학습과정은 다음의 colab notebook을 참조하여 진행하였습니다
    - https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
    
<br>

## 발견한 점
- 정확도가 0.98수준으로 매우 높은 분류 정확도를 보였습니다
- KoBERT가 본 task에 적합한 모델임을 확인했습니다. 따라서 BERT기반의 모델을 생성할 것으로 결정하였습니다
- 다만 본 notebook은 위의 url의 내용을 상당히 참조하였으므로 보다 간결하고 명확한 코드 구성을 위하여 수정된 모델을 생성할 필요성도 느꼈습니다

In [1]:
import os, sys
from google.colab import drive

#내 구글드라이브 디렉토리 설정
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 package_collection 폴더에 패키지 저장
#매 실행마다 이 코드를 재실행해야 해당 디렉토리에 있는 패키지를 사용할 수 있음
os.symlink('/content/drive/MyDrive/Colab Notebooks/py_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [2]:
#https://github.com/SKTBrain/KoBERT/issues/38
#해당 이슈가 발생하여 다음의 작업을 시도
#원래 폴더에 있던 모듈들을 모두 지우고 다시 설치한것임
!pip install --target=$my_path mxnet
!pip install --target=$my_path gluonnlp pandas tqdm
!pip install --target=$my_path sentencepiece
!pip install --target=$my_path transformers==3
!pip install --target=$my_path git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 55.0MB 52kB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 14.5MB 251kB/s 
     |████████████████████████████████| 143kB 59.0MB/s 
     |████████████████████████████████| 153kB 59.5MB/s 
     |████████████████████████████████| 184kB 58.1MB/s 
     |████████████████████████████████| 61kB 11.1MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 348kB 8.4MB/s 
     |████████████████████████████████| 9.5MB 11.2MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
  Using cached https://files.pythonhosted.org/packages/87/86/753182c9085ba4936c0076269a571613387cdb77ae2bf537448bfd63472c/numpy-1.19.4-cp36-cp36m-manylinux2010_x86_64.whl
     |████████████████████████████████| 2.0MB 57.2MB/s 
     |████████████████████████████████| 235kB 50.6MB/s 
     |████████████████████████████████| 512kB 44.6MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588513 sha256=0f3af968e0c56527e5f6a77b85a7a062352780cbb628ed6adf44ccae1b409e40
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
ERROR: datascienc

     |████████████████████████████████| 1.1MB 8.1MB/s 
     |████████████████████████████████| 757kB 9.3MB/s 
     |████████████████████████████████| 727kB 16.2MB/s 
  Using cached https://files.pythonhosted.org/packages/87/86/753182c9085ba4936c0076269a571613387cdb77ae2bf537448bfd63472c/numpy-1.19.4-cp36-cp36m-manylinux2010_x86_64.whl
     |████████████████████████████████| 3.0MB 26.0MB/s 
  Using cached https://files.pythonhosted.org/packages/e5/2d/6d4ca4bef9a67070fa1cac508606328329152b1df10bdf31fb6e4e727894/sentencepiece-0.1.94-cp36-cp36m-manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b1/a7/588bfa063e7763247ab6f7e1d994e331b85e0e7d09f853c59a6eb9696974/packaging-20.8-py2.py3-none-any.whl
     |████████████████████████████████| 890kB 45.2MB/s 
  Using cached https://files.pythonhosted.org/packages/8a/54/115f0c28a61d56674c3a5e05c46d6c3523ad196e1dcd3e2d8b119026df36/tqdm-4.54.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/c

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-1cvteg0c
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-1cvteg0c
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=7a7fa7b5181e88044ef679ab5287c11c1ebd8b0568e13900b9c47599029bd04d
  Stored in directory: /tmp/pip-ephem-wheel-cache-jxjc95mx/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [18]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/dacon/news_train.csv')

import re

stopwords_kr = pd.read_csv('/content/drive/MyDrive/dacon/stopwords_kr.txt',sep = '\n', header = None)
stop_kr_list = list(stopwords_kr.iloc[:,0])

url_with_https = re.compile(r'(https?:\/\/){1,}([\w.\/%-]+){1,}')
rm_email = re.compile(r'([a-zA-Z0-9\.\_]+)@([\w\.]+){1,}')
useless_words = re.compile(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]+')
rm_float_num = re.compile(r'(\d+\.?\,?\d+)+')
rm_num = re.compile(r'\d')

#지난 여름 자율프로젝트에서 사용했던 텍스트 전처리 방식 가져옴
def clean_kor_text(kor_text):
    text = url_with_https.sub('',kor_text)
    text = rm_email.sub('', text)
    text = useless_words.sub('',text)
    text = rm_float_num.sub('',text)
    text = rm_num.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stop_kr_list)
    return text

train_df['clean_content'] = train_df['content'].apply(clean_kor_text)

In [20]:
dataframe = train_df[['clean_content', 'info']]

In [43]:
dataframe.to_csv('/content/drive/MyDrive/dacon/for_glnlp_no_cls.csv', index = False)

In [20]:
dataframe.to_csv('/content/drive/MyDrive/dacon/for_glnlp_no_cls.txt',index = False)

In [7]:
train_dataset = nlp.data.TSVDataset('/content/drive/MyDrive/dacon/for_glnlp_no_cls.txt', num_discard_samples= 1,
                                    field_separator = lambda x: x.split(','))

In [8]:
train_dataset[0]

['이데일리 MARKETPOINT 현재 코스닥 기관 억 순매도', '0']

In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [10]:
word_to_id = vocab.token_to_idx
id_to_word = vocab.idx_to_token

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
tok(train_dataset[0][0])

['▁이데일리',
 '▁M',
 'AR',
 'K',
 'E',
 'T',
 'P',
 'O',
 'IN',
 'T',
 '▁현재',
 '▁코스닥',
 '▁기관',
 '▁',
 '억',
 '▁순매도']

In [14]:
data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

In [40]:
#훈련, 검증 데이터셋 분리
len(data_train)

118745

In [41]:
0.9* len(data_train)

106870.5

In [15]:
train_dataset, val_dataset = torch.utils.data.random_split(data_train, [106870, len(data_train) - 106870],
                                                           torch.Generator().manual_seed(156))

In [85]:
len(train_dataset)

106870

In [16]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, num_workers = 5)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size= batch_size, num_workers = 5)

In [17]:
train_dataloader.dataset[0]

(array([   2,  517, 5330, 5646,  517, 5435, 5940, 5760, 4617, 7310, 4663,
        7565, 4955, 4162, 5336, 7086, 4213, 6180,  517, 5330, 6353, 7010,
        1681, 5935, 5402, 3939, 2573, 7483, 7115, 2822, 6280, 7828, 5025,
        7112,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(35, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [20]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
#옵티마이저 및 손실함수 생성
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [22]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [23]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [26]:
for epoch in range(num_epochs):
  train_acc = 0.0
  val_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    loss = loss_fn(out, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    train_acc += calc_accuracy(out, label)
    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {} train acc {}".format(epoch+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
  print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))
  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    val_acc += calc_accuracy(out, label)
  print("epoch {} val acc {}".format(epoch+1, val_acc / (batch_id+1)))
    #2 epoch부터 정확도가 0.99에 수렴해감

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.9476277232170105 train acc 0.296875
epoch 1 batch id 201 loss 0.12110130488872528 train acc 0.8346548507462687
epoch 1 batch id 401 loss 0.1252095252275467 train acc 0.9008338528678305
epoch 1 batch id 601 loss 0.046764712780714035 train acc 0.9251507903494176
epoch 1 batch id 801 loss 0.07999251037836075 train acc 0.9374024656679151
epoch 1 batch id 1001 loss 0.11463430523872375 train acc 0.9460383366633367
epoch 1 batch id 1201 loss 0.06347007304430008 train acc 0.9514597210657785
epoch 1 batch id 1401 loss 0.003913508728146553 train acc 0.9553778551034975
epoch 1 batch id 1601 loss 0.0036750147119164467 train acc 0.9593125390381012

epoch 1 train acc 0.9603650338212464


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 val acc 0.9844734062980031


epoch 2 batch id 1 loss 0.007401120848953724 train acc 1.0
epoch 2 batch id 201 loss 0.02528110146522522 train acc 0.9862406716417911
epoch 2 batch id 401 loss 0.12702900171279907 train acc 0.9868687655860349
epoch 2 batch id 601 loss 0.005864911712706089 train acc 0.9873648086522463
epoch 2 batch id 801 loss 0.00527837174013257 train acc 0.9878082084893882
epoch 2 batch id 1001 loss 0.006690469104796648 train acc 0.9884646603396603
epoch 2 batch id 1201 loss 0.0008331240969710052 train acc 0.9893968567860116
epoch 2 batch id 1401 loss 0.0008620749576948583 train acc 0.9898844575303355
epoch 2 batch id 1601 loss 0.0462048165500164 train acc 0.9904551842598376

epoch 2 train acc 0.990568862275449


...


epoch 3 val acc 0.991263440860215


epoch 4 batch id 1 loss 0.0018559291493147612 train acc 1.0
epoch 4 batch id 201 loss 0.0015884003369137645 train acc 0.997978855721393
epoch 4 batch id 401 loss 0.06874649971723557 train acc 0.9975451995012469
epoch 4 batch id 601 loss 0.003892687615007162 train acc 0.9974781613976705
epoch 4 batch id 801 loss 0.04626983404159546 train acc 0.9972885455680399
epoch 4 batch id 1001 loss 0.00018035073298960924 train acc 0.9975493256743256
epoch 4 batch id 1201 loss 0.0001671194186201319 train acc 0.9977883014154871
epoch 4 batch id 1401 loss 0.00019342500308994204 train acc 0.9978586723768736
epoch 4 batch id 1601 loss 0.00014779255434405059 train acc 0.9979504996876952

epoch 4 train acc 0.9980164670658682



epoch 4 val acc 0.9919354838709677


epoch 5 batch id 1 loss 0.0009059857111424208 train acc 1.0
epoch 5 batch id 201 loss 0.0027909823693335056 train acc 0.9990671641791045
epoch 5 batch id 401 loss 0.0030444017611443996 train acc 0.9989869077306733
epoch 5 batch id 601 loss 0.00019194385095033795 train acc 0.9989600665557404
epoch 5 batch id 801 loss 0.0001697242696536705 train acc 0.9989271223470662
epoch 5 batch id 1001 loss 0.00013839890016242862 train acc 0.999000999000999
epoch 5 batch id 1201 loss 0.00013379621668718755 train acc 0.9990502706078268
epoch 5 batch id 1401 loss 0.00013402276090346277 train acc 0.999040863668808
epoch 5 batch id 1601 loss 0.0001387248485116288 train acc 0.9990338069956277

epoch 5 train acc 0.9990737275449102



epoch 5 val acc 0.9919354838709677


In [27]:
#모델 저장
#https://honeyjamtech.tistory.com/39

def save_checkpoint(epoch, model, optimizer, filename):
  state = {
      'Epoch' : epoch,
      'State_dict' : model.state_dict(),
      'optimizer' : optimizer.state_dict()
  }
  torch.save(state, filename)

save_checkpoint(num_epochs, model, optimizer, '/content/drive/MyDrive/dacon/bert_1219.pt')

In [28]:
save_checkpoint(num_epochs, model, optimizer, '/content/drive/MyDrive/dacon/bert_1219.pth')

In [30]:
#테스트 데이터 전처리 및 토큰화
import pandas as pd
import re

stopwords_kr = pd.read_csv('/content/drive/MyDrive/dacon/stopwords_kr.txt',sep = '\n', header = None)
stop_kr_list = list(stopwords_kr.iloc[:,0])

url_with_https = re.compile(r'(https?:\/\/){1,}([\w.\/%-]+){1,}')
rm_email = re.compile(r'([a-zA-Z0-9\.\_]+)@([\w\.]+){1,}')
useless_words = re.compile(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]+')
rm_float_num = re.compile(r'(\d+\.?\,?\d+)+')
rm_num = re.compile(r'\d')

#지난 여름 자율프로젝트에서 사용했던 텍스트 전처리 방식 가져옴
def clean_kor_text(kor_text):
    text = url_with_https.sub('',kor_text)
    text = rm_email.sub('', text)
    text = useless_words.sub('',text)
    text = rm_float_num.sub('',text)
    text = rm_num.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stop_kr_list)
    return text

test_df = pd.read_csv('/content/drive/MyDrive/dacon/news_test.csv')

test_df['clean_cont'] = test_df['content'].apply(clean_kor_text)

In [41]:
test_dataframe = test_df[['clean_cont', 'info']].to_csv('/content/drive/MyDrive/dacon/for_glnlp_no_cls_test.txt',index = False)

In [42]:
test_dataset = nlp.data.TSVDataset('/content/drive/MyDrive/dacon/for_glnlp_no_cls_test.txt', num_discard_samples= 1,
                                   field_separator = lambda x: x.split(','))

In [43]:
test_dataset[0]

['마이크로 LED TV 장비 양산 돌입 전방업체 투자 확대로 본업도 호조연일 신고가', '0']

In [44]:
data_test = BERTDataset(test_dataset, sent_idx = 0, label_idx= 1,bert_tokenizer= tok, max_len = max_len, pad = True, pair = False)

In [45]:
data_test[0]

(array([   2, 1907, 7096, 7565, 6079,  517,  312,  694, 3961, 3214, 6516,
        1738, 4012, 6305, 6879, 4768, 5125, 6079, 2408, 6873, 5859, 5093,
        6928, 7126, 3012, 5330,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(27, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [47]:
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size= batch_size, num_workers = 5, shuffle= False)

In [49]:
#테스트 데이터셋 평가

pred = []

model.eval() #모델을 평가모드로

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length

    with torch.no_grad(): #학습을 하지 않아야 하므로 꼭 명시해야함
      out = model(token_ids, valid_length, segment_ids) 

    logits = out.detach().cpu().numpy() #최종 산출물 
 
    pred.append(logits) #예측 리스트에 저장

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [54]:
print(len(pred)) #각 배치마다 output을 최고 값 기준으로 하도록

2228


In [55]:
print(len(pred[0]))

64


In [65]:
torch.Tensor(pred[0][0]).shape

torch.Size([2])

In [59]:
pred[0][0]

array([-1.0992062,  0.8840941], dtype=float32)

In [67]:
test_val, test_indices = torch.max(torch.Tensor(pred[0]), 1)

In [68]:
test_indices

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [74]:
#최종으로 최고값으로 예측
pred_result = []
for i in range(len(pred)):
  logits = pred[i]
  max_vals, max_indices = torch.max(torch.Tensor(pred[i]), 1)
  max_indices = max_indices.detach().cpu().numpy()
  pred_result.extend(max_indices)

In [75]:
len(pred_result)

142565

In [78]:
type(pred_result[0])

numpy.int64

In [79]:
#최종 submission_df에 합치기

test_submission = pd.read_csv('/content/drive/MyDrive/dacon/sample_submission.csv')
test_submission.head()

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0


In [80]:
test_submission['info'] = pred_result

In [84]:
test_submission.to_csv('/content/drive/MyDrive/dacon/mh_second_submission_bert.csv', index = False)
  #최종 제출 결과 0.98의 정확도가 산출됨